In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2
from pathlib import Path
from faker import Faker
from collections import OrderedDict
from datetime import datetime
import time
from insert_class import inserare_date
import random
import math
import getpass

In [2]:
fake = Faker()
min_hour = 7
max_hour = 21
sample_zi_de_votare = 0.0628


In [3]:
h, d, u, p = "localhost", "staging", input("User:"), getpass.getpass('Password:')

User: postgres
Password: ········


In [4]:
def batch_size_sum_sample(n, total):

    dividers = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]

In [5]:
if __name__ == "__main__":

    app = inserare_date(h, d, u, p)
    df = app.read_sql("staging.populatie")
    df = df.sample(frac=sample_zi_de_votare, weights=df.groupby('cod_judet')['cod_judet'].transform('count'))
    
    df_vd = pd.DataFrame(data={'CNP': df['cnp'].to_list()})
    df_vd['serialnbr'] = [i+1 for i in range(df_vd.shape[0])]

    df_vd['vot'] = df_vd.apply(lambda x: fake.random_elements(elements=OrderedDict([('BJP', .18), 
                                                                                    ('INC', .17), 
                                                                                    ('BSP', .17), 
                                                                                    ('CMD', .10), 
                                                                                    ('IPV', .10), 
                                                                                    ('BIT', .02), 
                                                                                    ('UTC', .06), 
                                                                                    ('VOP', .08), 
                                                                                    ('KLM', .09), 
                                                                                    ('JAR', .03)]), 
                                                              length=1),  
                               axis=1)
    
#     df_vd['vot'] = df_vd.apply(lambda x: str(fake.random_elements(elements=('BJP', 'INC', 'BSP', 'CMD', 'IPV', 'BIT', 'UTC', 'VOP', 'KLM', 'JAR'), unique=True, length=1))[2:-2], axis=1)

    
    df_vd['vot'] = df_vd['vot'].astype(str).str.replace("[", "").str.replace("]", "")
    df_vd['vot'] = df_vd['vot'].astype(object).str.replace("'", "")

    df_vd = df_vd[['serialnbr', 'CNP', 'vot']]
    df_vd['ora_votarii'] = np.nan

    batch_size = batch_size_sum_sample(14, df_vd.shape[0])

    for _hour, _bs in zip(range(min_hour, max_hour), batch_size) :

        temp_df_vd = df_vd[df_vd['ora_votarii'].isnull()]
        temp_df_vd = temp_df_vd.sample(n=_bs)
        temp_df_vd['ora_votarii'] = temp_df_vd['ora_votarii'].apply(lambda x: fake.date_time_between_dates(datetime(2021, 4, 11, _hour, 0, 0), datetime(2021, 4, 11, _hour+1, 0, 0)))

        df_vd = pd.merge(df_vd, temp_df_vd, 
                         on='CNP', how='left')
        df_vd['ora_votarii'] = np.where(df_vd['ora_votarii_x'].isnull(), df_vd['ora_votarii_y'].astype('datetime64[ns]'), df_vd['ora_votarii_x'].astype('datetime64[ns]'))
        df_vd.rename(columns={'serialnbr_x': 'serialnbr',
                              'vot_x': 'vot'}, inplace=True)
        df_vd = df_vd[['serialnbr', 'CNP', 'vot', 'ora_votarii']]

        print(f"hour: {_hour} / batch size: {_bs}")
    
    app.con_close()
    



hour: 7 / batch size: 15032
hour: 8 / batch size: 146
hour: 9 / batch size: 381
hour: 10 / batch size: 25963
hour: 11 / batch size: 15748
hour: 12 / batch size: 13475
hour: 13 / batch size: 6247
hour: 14 / batch size: 175
hour: 15 / batch size: 15626
hour: 16 / batch size: 7781
hour: 17 / batch size: 7719
hour: 18 / batch size: 3527
hour: 19 / batch size: 784
hour: 20 / batch size: 27797


In [16]:
app = inserare_date(h, d, u, p)
app.send_query("""DELETE FROM staging.ziua_votarii;""")

for i in range(min_hour, max_hour):
    print(f"hour: {i}")
    df_vd_insert = df_vd[df_vd['ora_votarii'].apply(lambda x: x.hour)==i]
    app.insert_into(df_vd_insert, 'staging.ziua_votarii', batch_size=1000, expand=True)
    
#     time.sleep(1)
app.con_close()

Query `DELETE FROM staging.ziua_votarii;` returned successfully
hour: 7
before: (0,)
after:: (1000,)
before: (1000,)
after:: (2000,)
before: (2000,)
after:: (3000,)
before: (3000,)
after:: (4000,)
before: (4000,)
after:: (5000,)
before: (5000,)
after:: (6000,)
before: (6000,)
after:: (7000,)
before: (7000,)
after:: (8000,)
before: (8000,)
after:: (9000,)
before: (9000,)
after:: (10000,)
before: (10000,)
after:: (11000,)
before: (11000,)
after:: (12000,)
before: (12000,)
after:: (13000,)
before: (13000,)
after:: (14000,)
before: (14000,)
after:: (15000,)
before: (15000,)
after:: (15029,)
hour: 8
before: (15029,)
after:: (15178,)
hour: 9
before: (15178,)
after:: (15559,)
hour: 10
before: (15559,)
after:: (16559,)
before: (16559,)
after:: (17559,)
before: (17559,)
after:: (18559,)
before: (18559,)
after:: (19559,)
before: (19559,)
after:: (20559,)
before: (20559,)
after:: (21559,)
before: (21559,)
after:: (22559,)
before: (22559,)
after:: (23559,)
before: (23559,)
after:: (24559,)
before: